In [1]:
import numpy as np
import pandas as pd
import itertools
import sys
import copy

In [2]:
header = ['name', 'salary', 'value']

data = [
['Иван', 800, 15],
['Петр', 1700,29],
['Михаил', 400, 7],
['Светлана',  900, 18],
['Евгений',1100, 22],
['Надежда',  2200, 32],
['Ирина',  600, 14],
['Максим',  1400, 24],
['Анна',  1700, 30],
['Матвей',  500, 8],
['Ян',  200,  3],
['Ника',  1300,  22],
['Ольга',  800,  16],
['Константин',  1900, 28],
['Алиса',  300, 5],
['Андрей', 1500, 21],
['Павел',  300, 5]]

In [3]:
frame = pd.DataFrame(data, columns=header)
frame.head()

,name,salary,value
0,Иван,800,15
1,Петр,1700,29
2,Михаил,400,7
3,Светлана,900,18
4,Евгений,1100,22


# Brute Force

In [4]:
data = frame.iloc[:,1:3].values

In [5]:
def best_choise_brute_force(data, weight_full = 7000):
    """
    Choose the best stuff with brute force algorithm.
    
    Parameters
    -------
        data: np.array (N, 2)
            The array has the next structure: [[weight, value], ..].
        weight_full: int
            Max weihgt of the "bag"
    
    Returns
    -------
        permitation: array (, 2)
            The array has the next structure: [[value, indexis], ..].
            
    """
    permitation = []
    N = len(data)
    
    #All possible choices (2**N)
    perm = ["".join(seq) for seq in itertools.product("01", repeat=N)]

    for row in perm:   
        #str -> array of numbers
        choice = [int(i) for i in row]
        
        #count the weight and value of the selected objects
        weight = np.dot(choice, data[:, 0])
        value = np.dot(choice, data[:, 1])
        
        if weight <= weight_full:
            permitation.append([value, row])
    
    return permitation

In [6]:
%%time
permitation = best_choise_brute_force(data)

CPU times: user 2 s, sys: 25.4 ms, total: 2.02 s
Wall time: 2 s


In [7]:
df_perm = pd.DataFrame(permitation)

In [8]:
#sort by value
df_perm[0] *= -1
df_perm.sort_values(by=[0], inplace=True)
df_perm[0] *= -1

In [9]:
df_perm.head()

,0,1
28790,133,10111010111010000
24433,133,10011010110010101
28771,132,10111010100010101
7140,132,00011011101010100
32102,132,11111010011010000


In [10]:
# Seelct maximum values
best_choise = df_perm[df_perm[0] == df_perm[0].max()][1].values
print(best_choise)

['10111010111010000' '10011010110010101']


In [11]:
# transform strings to the boolean array for further processing
best_choise_nask = [ [ bool(int(el)) for el in row] for row in best_choise]

In [12]:
print(best_choise_nask)

[[True, False, True, True, True, False, True, False, True, True, True, False, True, False, False, False, False], [True, False, False, True, True, False, True, False, True, True, False, False, True, False, True, False, True]]


In [13]:
#Select persons with our mask
best_frame_1 = frame[best_choise_nask[0]]
best_frame_2 = frame[best_choise_nask[1]]

In [14]:
best_frame_1

,name,salary,value
0,Иван,800,15
2,Михаил,400,7
3,Светлана,900,18
4,Евгений,1100,22
6,Ирина,600,14
8,Анна,1700,30
9,Матвей,500,8
10,Ян,200,3
12,Ольга,800,16


In [15]:
#Check
best_frame_1.iloc[:,1:3].sum()

salary    7000
value      133
dtype: int64

In [16]:
best_frame_2

,name,salary,value
0,Иван,800,15
3,Светлана,900,18
4,Евгений,1100,22
6,Ирина,600,14
8,Анна,1700,30
9,Матвей,500,8
12,Ольга,800,16
14,Алиса,300,5
16,Павел,300,5


In [17]:
#Check
best_frame_1.iloc[:,1:3].sum()

salary    7000
value      133
dtype: int64

# Dynamic Programming/  Knapsack Problem

In [18]:
frame.head()

,name,salary,value
0,Иван,800,15
1,Петр,1700,29
2,Михаил,400,7
3,Светлана,900,18
4,Евгений,1100,22


We can devide our 'salary' column by 100 to do the dimension of our  'Bags" table smaller. 
It's possible because we haven't got numbers in 'salary' column no multiples 100.

In [19]:
frame_100 = copy.deepcopy(frame)
frame_100['salary'] //= 100

In [20]:
weight_full = 7000 // 100
data_100 = frame_100.iloc[:,1:3].values

In [88]:
def best_choise(data, weight_full = 70):
    """
    Choose the best stuff with dynamic programming.
    Knapsack Problem.
    
    Parameters
    -------
        data: np.array (N, 2)
            The array has the next structure: [[weight, value], ..].
        weight_full: int
            Max weihgt of the "bag"
    
    Returns
    -------
        list_out: array
            Array with indexes of the best objects.
            
    """
    
    #wt - 'weights' of the objects
    #val - 'values' of the objects
    
    #append nothing to the start of the arrays to have zeros at 
    #the first column and first row of our matrix V
    wt = np.concatenate(([0],data_100[:,0]))
    val = np.concatenate(([0],data_100[:,1]))

    #weight_full + 1 . Because [0, 70) doesn't include 70
    V = np.zeros((len(wt), weight_full + 1))
    
    #build our matrix
    for i in range(V.shape[0]):
        for w in range(V.shape[1]):
            if i != 0:
                if wt[i] > w:
                    V[i, w] = V[i-1, w]
                elif wt[i] <= w:
                    V[i,w] = max(V[i-1, w] , val[i] + V[i-1, w - wt[i]])
    
    max_val = V[-1,-1]
    
    i = V.shape[0] - 1
    w = V.shape[1] - 1
    list_out = []
    
    #find indexes the best objects
    while i > 0 and w > 0:
        if V[i, w] != V[i-1, w]:
            list_out.append(i - 1)
            w -= wt[i]
        i -= 1
                    
    return list_out
    

In [89]:
best_idxs = best_choise(data_100)

In [90]:
best_frame = frame.iloc[best_idxs, :]
best_frame

,name,salary,value
12,Ольга,800,16
10,Ян,200,3
9,Матвей,500,8
8,Анна,1700,30
6,Ирина,600,14
4,Евгений,1100,22
3,Светлана,900,18
2,Михаил,400,7
0,Иван,800,15


In [91]:
best_frame.iloc[:,1:3].sum()

salary    7000
value      133
dtype: int64